In [0]:
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=b6c0d1e6c2f274f13551761b1030e986515b1e600524661bc94905fb6d14bbd0
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=b3e9628950294a03023bb7edb0b4ff3459660b5a9590c651a6d6831605ec2cf7
  Stored in directory: /root/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=ed10c8ff8c6008dd9d2a55544e7040491d02082f3cd56a60941c6998be74e707
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=e407aa65eeaf2ec30e425808f01770971845a9149d84250a9caf99235317bf08
  Stor

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir -p ~/.kaggle
import json 
token = {"username":"seunghwan1228","key":"2a8fb35ecd319bf10a02713c306059ca"}

with open('/content/drive/My Drive/Colab Notebooks/Kaggle Kernel/kaggle.json', 'w') as file:
  json.dump(token, file)
 

!cp drive/'My Drive'/'Colab Notebooks'/'Kaggle Kernel'/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json

# down load path
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification
!unzip train.csv.zip

kaggle.json
total 4
-rw------- 1 root root 72 Nov  3 08:07 kaggle.json
{"username": "seunghwan1228", "key": "2a8fb35ecd319bf10a02713c306059ca"}Warning: Looks like you're using an outdated API Version, please consider updating (server 1.5.6 / client 1.5.4)
100% 273M/273M [00:04<00:00, 70.2MB/s]

 75% 9.00M/12.0M [00:00<00:00, 27.6MB/s]
100% 12.0M/12.0M [00:00<00:00, 30.3MB/s]
  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 190MB/s]
Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
!wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip

--2019-11-03 08:29:35--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1248381879 (1.2G) [application/zip]
Saving to: ‘wwm_uncased_L-24_H-1024_A-16.zip’

wwm_uncased_L-24_H- 100%[===================>]   1.16G   175MB/s    in 7.2s    

2019-11-03 08:29:42 (165 MB/s) - ‘wwm_uncased_L-24_H-1024_A-16.zip’ saved [1248381879/1248381879]



In [0]:
!unzip /content/wwm_uncased_L-24_H-1024_A-16.zip

Archive:  /content/wwm_uncased_L-24_H-1024_A-16.zip
   creating: wwm_uncased_L-24_H-1024_A-16/
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_config.json  


In [0]:
import numpy as np
import pandas as pd
import os
import sys
import keras
import tensorflow as tf
from keras_bert import tokenizer
import keras_bert
from tqdm import tqdm

In [0]:
df = pd.read_csv('/content/train.csv')

In [0]:
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [0]:
text_data = df['comment_text']
target = df['target']

In [0]:
def target_binary(x):
  if x >= 0.5:
    return 1
  else:
    return 0

In [0]:
target = target.apply(target_binary)

In [0]:
display(text_data.head())
display(target.head())

0    This is so cool. It's like, 'would you want yo...
1    Thank you!! This would make my life a lot less...
2    This is such an urgent design problem; kudos t...
3    Is this something I'll be able to install on m...
4                 haha you guys are a bunch of losers.
Name: comment_text, dtype: object

0    0
1    0
2    0
3    0
4    1
Name: target, dtype: int64

In [0]:
import keras_bert

In [0]:
pretrained_path = '/content/wwm_uncased_L-24_H-1024_A-16'

config_path = '/content/wwm_uncased_L-24_H-1024_A-16/bert_config.json'

check_point_path = '/content/wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001'

vocab_path = '/content/wwm_uncased_L-24_H-1024_A-16/vocab.txt'

In [0]:
DATA_COLUMN = 'comment_text'
LABEL_COLUMN = 'target'

SEQ_LEN = 128

In [0]:
# get tokenizer

token_dict = keras_bert.load_vocabulary(vocab_path)

In [0]:
tokenizer = keras_bert.Tokenizer(token_dict)

In [0]:
tokenizer.tokenize('hello bert tokenizer &')

['[CLS]', 'hello', 'bert', 'token', '##izer', '&', '[SEP]']

In [0]:
tokenizer

In [0]:
def convert_data(data_df):
  global tokenizer
  indices, targets= [], []
  for i in tqdm(range(len(data_df))):
    ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
    indices.append(ids)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].apply(target_binary)
    targets.append(data_df[LABEL_COLUMN][i])
  items = list(zip(indices, targets))
  np.random.shuffle(items)
  indices, targets = zip(*items)
  indices = np.array(indices)
  return [indices, np.zeros_like(indices)], np.array(targets)

In [0]:
def load_data(path):
  data_df= pd.read_csv(path, nrows=10000)
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype('str')
  data_x, data_y = convert_data(data_df)
  return data_x, data_y

In [0]:
train_x, train_y = load_data('/content/train.csv')

100%|██████████| 10000/10000 [00:52<00:00, 191.26it/s]


In [0]:
print(train_x)
print(len(train_x))
print(train_x[0].shape)

[array([[  101,  2056,  1996, ...,     0,     0,     0],
       [  101, 22889,  2819, ...,     0,     0,     0],
       [  101,  1996,  2373, ...,     0,     0,     0],
       ...,
       [  101,  7473,  1012, ...,  2154,  2028,   102],
       [  101,  1000,  1000, ...,  1012,  2096,   102],
       [  101,  2057,  2064, ...,     0,     0,     0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])]
2
(10000, 128)


In [0]:
print(train_y)
print(len(train_y))
print(train_y.shape)

[0 0 0 ... 0 0 0]
10000
(10000,)


## load model

In [0]:
model = keras_bert.get_model(len(token_dict), seq_len=128)

In [0]:
bert_input = model.inputs[:2]

print(bert_input)

[<tf.Tensor 'Input-Token_1:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'Input-Segment_1:0' shape=(?, 128) dtype=float32>]


In [0]:
bert_output = model.layers[-3].output

In [0]:
real_output = keras.layers.Dense(1, activation='sigmoid')(bert_output)

In [0]:
real_output

<tf.Tensor 'dense_2/Sigmoid:0' shape=(?, 1) dtype=float32>

In [0]:
bert_model = keras.models.Model(bert_input, real_output)

In [0]:
bert_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 128)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 128)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
bert_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [0]:
train_x

[array([[  101,  2056,  1996, ...,     0,     0,     0],
        [  101, 22889,  2819, ...,     0,     0,     0],
        [  101,  1996,  2373, ...,     0,     0,     0],
        ...,
        [  101,  7473,  1012, ...,  2154,  2028,   102],
        [  101,  1000,  1000, ...,  1012,  2096,   102],
        [  101,  2057,  2064, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [0]:
train_y

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
bert_model.fit(train_x, train_y, epochs=10, batch_size=32)

Epoch 1/10
10000/10000 [==============================] - 520s 52ms/step - loss: 0.2845 - acc: 0.9374
Epoch 2/10
10000/10000 [==============================] - 519s 52ms/step - loss: 0.2279 - acc: 0.9422
Epoch 3/10
10000/10000 [==============================] - 519s 52ms/step - loss: 0.2253 - acc: 0.9422
Epoch 4/10
10000/10000 [==============================] - 519s 52ms/step - loss: 0.2301 - acc: 0.9422
Epoch 5/10
10000/10000 [==============================] - 519s 52ms/step - loss: 0.2298 - acc: 0.9422
Epoch 6/10
10000/10000 [==============================] - 519s 52ms/step - loss: 0.2274 - acc: 0.9422
Epoch 7/10
10000/10000 [==============================] - 519s 52ms/step - loss: 0.2270 - acc: 0.9422
Epoch 8/10
10000/10000 [==============================] - 519s 52ms/step - loss: 0.2276 - acc: 0.9422
Epoch 9/10
10000/10000 [==============================] - 519s 52ms/step - loss: 0.2297 - acc: 0.9422
Epoch 10/10
10000/10000 [==============================] - 519s 52ms/step - loss: 

In [0]:
def convert_test(test_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(test_df))):
        ids, segments = tokenizer.encode(test_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]
  
def load_test(data_df):
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = convert_test(data_df)
    return data_x

In [0]:
test_sentence = ['this movie is so trash']


test_sentence = pd.DataFrame({DATA_COLUMN:test_sentence})

In [0]:
test_x = load_test(test_sentence)

100%|██████████| 1/1 [00:00<00:00, 402.02it/s]


In [0]:
test_x

[array([[  101,  2023,  3185,  2003,  2061, 11669,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,

In [0]:
bert_model.predict(convert_test(test_sentence)[0])

100%|██████████| 1/1 [00:00<00:00, 1231.08it/s]


ValueError: ignored

In [0]:
# original bert model


model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 128)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 128)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________